In [1]:

import numpy as np
#@save
import collections
import hashlib
import inspect
import math
import os
import random
import re
import shutil
import sys
import tarfile
import time
import zipfile
from collections import defaultdict
import pandas as pd
import requests
from IPython import display
from matplotlib import pyplot as plt
from matplotlib_inline import backend_inline

d2l = sys.modules[__name__]

#@save
import numpy as np
import torch
import torchvision
from PIL import Image
from scipy.spatial import distance_matrix
from torch import nn
from torch.nn import functional as F
from torchvision import transforms

import numpy as np
#@save
import collections
import hashlib
import inspect
import math
import os
import random
import re
import shutil
import sys
import tarfile
import time
import zipfile
from collections import defaultdict
import pandas as pd
import requests
from IPython import display
from matplotlib import pyplot as plt
from matplotlib_inline import backend_inline

d2l = sys.modules[__name__]

#@save
import numpy as np
import torch
import torchvision
from PIL import Image
from scipy.spatial import distance_matrix
from torch import nn
from torch.nn import functional as F
from torchvision import transforms
import d2l.torch as d2l


In [3]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

X = torch.rand(2, 20)
net(X).shape

torch.Size([2, 10])

In [4]:
class MLP(nn.Module):
    def __init__(self):
        # Call the constructor of the parent class nn.Module to perform
        # the necessary initialization
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.out = nn.LazyLinear(10)

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input X
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [5]:
net = MLP()
net(X).shape

torch.Size([2, 10])

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self.add_module(str(idx), module)

    def forward(self, X):
        for module in self.children():
            X = module(X)
        return X

In [7]:
net = MySequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
net(X).shape

torch.Size([2, 10])

In [9]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20))
        self.linear = nn.LazyLinear(20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(X @ self.rand_weight + 1)
        # Reuse the fully connected layer. This is equivalent to sharing
        # parameters with two fully connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [10]:
net = FixedHiddenMLP()
net(X)

tensor(-0.0372, grad_fn=<SumBackward0>)

In [11]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.LazyLinear(64), nn.ReLU(),
                                 nn.LazyLinear(32), nn.ReLU())
        self.linear = nn.LazyLinear(16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.LazyLinear(20), FixedHiddenMLP())
chimera(X)

tensor(-0.2557, grad_fn=<SumBackward0>)

In [15]:
class ParallelBlock(nn.Module):
    def __init__(self, *args):
        super().__init__()
        self.linear = nn.LazyLinear(10)
        self.input_net = nn.ModuleList(args)

    def forward(self, X, X2):
        X = self.input_net[0](X)
        X2 = self.input_net[1](X2)
        return self.linear(torch.cat((X, X2), dim=1))

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(64)
        self.output = nn.LazyLinear(10)

    def forward(self, X):
        return self.output(F.relu(self.hidden(X)))

p = ParallelBlock(MLP(), MLP())
p(X, torch.rand(2, 20))

tensor([[ 0.0075, -0.0612, -0.0535,  0.2503, -0.2105, -0.1544, -0.0719, -0.0567,
         -0.0281,  0.2162],
        [-0.0123, -0.0898, -0.0464,  0.3037, -0.1755, -0.1518, -0.0579, -0.0434,
         -0.0300,  0.2483]], grad_fn=<AddmmBackward0>)

# parameter management

In [16]:
net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))

X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

In [17]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[-0.2987, -0.2062, -0.1837,  0.1151, -0.0132,  0.1164,  0.1512,  0.0332]])),
             ('bias', tensor([-0.1863]))])

In [18]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([-0.1863]))

In [22]:
net[2].weight.grad == None

True

In [23]:
[ (name, params) for name, params in net[2].named_parameters() ]

[('weight',
  Parameter containing:
  tensor([[-0.2987, -0.2062, -0.1837,  0.1151, -0.0132,  0.1164,  0.1512,  0.0332]],
         requires_grad=True)),
 ('bias',
  Parameter containing:
  tensor([-0.1863], requires_grad=True))]

In [25]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


# Params Init

In [27]:
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(), nn.LazyLinear(1))

X = torch.rand(size=(2, 4))

net(X).shape

torch.Size([2, 1])

In [28]:
def init_normal(module):
    if type(module) == nn.Linear:
        nn.init.normal_(module.weight, mean=0, std=0.01)
        nn.init.zeros_(module.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0042, -0.0060,  0.0021, -0.0134]), tensor(0.))

In [29]:
def init_constant(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 1)
        nn.init.zeros_(module.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [30]:
def init_xavier(module):
    if type(module) == nn.Linear:
        nn.init.xavier_uniform_(module.weight)

def init_42(module):
    if type(module) == nn.Linear:
        nn.init.constant_(module.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.1276,  0.6352,  0.1433,  0.3394])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [34]:
# curstom init
def my_init(module):
    if type(module) == nn.Linear:
        print(f"Init {module}.")
        with torch.no_grad():
            module.weight.uniform_(-10, 10)
            module.weight *= module.weight.abs() >= 5

net.apply(my_init)
print("Initialized weights:")
for name, param in net.named_parameters():
    if 'weight' in name:
        print(name, param)

Init Linear(in_features=4, out_features=8, bias=True).
Init Linear(in_features=8, out_features=1, bias=True).
Initialized weights:
0.weight Parameter containing:
tensor([[-9.7177,  5.0229, -8.8263,  0.0000],
        [-6.2265,  8.2088,  5.5343, -0.0000],
        [ 0.0000,  0.0000, -0.0000, -9.8167],
        [ 0.0000, -0.0000,  0.0000, -0.0000],
        [ 7.5310, -6.6718,  8.4652,  7.8925],
        [-0.0000, -6.4802,  0.0000,  5.8744],
        [-0.0000,  8.2700, -6.6113,  5.9287],
        [-8.3137,  0.0000,  8.9101,  7.2051]], requires_grad=True)
2.weight Parameter containing:
tensor([[ 8.0157, -0.0000, -5.9979,  6.2696,  0.0000, -0.0000,  0.0000, -9.9195]],
       requires_grad=True)
